<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>


# Эффект Доплера
### Вычисление скорости поезда с помощью Фурье-анализа звукового файла
### Example - Waves and Acoustics
By Magnus A. Gjennestad, Vegard Hagen, Aksel Kvaal, Morten Vassvik, Trygve B. Wiig and Peter Berg.

Last edited: 14th of May 2018.
___

Вы, вероятно, знакомы с эффектом Доплера. Возможно, вы замечали его как изменение тона автомобильного гудка, сирены скорой помощи или свистка поезда, когда машина/скорая помощь/поезд проезжали мимо вас. В научных терминах это изменение высоты тона описывается как сдвиг частоты звука.

Уравнение для звуковой частоты $f$, наблюдаемой приемником, движущимся со скоростью $v_r$ относительно воздуха, имеет вид

\begin{equation}
f = \frac{c+v_r}{c+v_s}f_0,
\label{1} \quad (1)
\end{equation}

где звук первоначально излучается с частотой $f_0$ источником, который движется со скоростью $v_s$ относительно воздуха. Здесь $c$ - это скорость звука.

Мы фокусируемся на движении вдоль прямой линии, где мы определяем $v_r$ как положительную, когда приемник движется к источнику, и отрицательную, когда источник удаляется от наблюдателя.

Классическим применением (1) является расчет скорости поезда, движущегося мимо стационарного приемника. Пусть наблюдаемая частота свиста поезда, когда поезд движется к приемнику, равна $f_1$, а наблюдаемая частота, когда поезд удаляется от приемника, равна $f_2$.

В этом случае мы можем использовать (1) для получения выражений для $f_1$ и $f_2$:
$$
\begin{align}
f_1&= \frac{c}{c-v}f_0,\qquad\qquad\qquad\qquad\\
f_2&= \frac{c}{c+v}f_0,\qquad\qquad\qquad\qquad
\end{align}
$$
где $v$ - это скорость поезда. Мы используем эти два уравнения для устранения неизвестного $f_0$ и решаем полученное уравнение для $v$

\begin{equation}
v = \frac{f_1 −f_2}{f_1+f_2}c.
\label{2} \quad (2)
\end{equation}

Теперь мы можем определить скорость поезда $v$, используя (2). Однако здесь будет небольшой поворот. Мы проанализируем численно экспериментальные данные в виде записи звука проходящего поезда, чтобы вычислить $f_1$ и $f_2$. Впоследствии это приведет к получению $v$.

Для этого мы будем использовать реализацию мощного инструмента, известного как быстрое преобразование Фурье (FFT), из библиотеки __numpy__. По сути, это не что иное, как эффективный способ вычисления дискретного преобразования Фурье (DFT), дискретного приближения непрерывного преобразования Фурье.

### Дискретное преобразование Фурье

Звуковой файл представлен в python в виде вектора $\vec{x}$ с элементами $N$, где каждый элемент представляет собой амплитуду звука, дискретизированную через временные интервалы $\Delta t$. DFT $\vec{x}$ тогда также является вектором с $N$ элементами. Мы называем его $\vec{X}$.
Предположим теперь, что мы знаем $\vec{X}$. Затем мы можем вычислить каждый элемент $x_n$ в $\vec{x}$, применив формулу
$$x_n = \frac{1}{N} \sum_{k=0}^{N-1}X_k \exp\left(i2\pi\frac{k}{N\Delta t}n\Delta t\right).$$
Давайте присмотримся к этому выражению поближе и попытаемся понять, что оно означает. Оно говорит нам о том, что $\vec{x}$ представляет собой суперпозицию экспоненциальных функций с различными частотами $f_k = \frac{k}{N\Delta t}$ и амплитудами $X_k$. Поэтому мы можем рассматривать величину амплитуд $|X_k|^2$ как меру "веса частоты $f_k$" в $\vec{x}$!

Q: Как мы вычисляем $\vec{X}$?

A: Мы могли бы применить формулу для дискретного преобразования Фурье,

$$X_k =\sum_{n=0}^{N-1}x_n \exp\left(-i2\pi\frac{k}{N\Delta t}n\Delta t\right).$$

Для этого требуется $\mathcal{O}(N^2)$ операций. Напротив, FFT является более вычислительно эффективным способом вычисления $\vec{X}$, требующим только $\mathcal{O}(N \ln N)$ операций. Существует несколько алгоритмов БПФ, и многие из них используют тот факт, что все экспоненты могут быть записаны как

$$\left(\exp\left(-\frac{2\pi i}{N}\right)\right)^{kn}$$

В python вы можете вычислить $\vec{x}$ из $\vec{X}$ с помощью <span style="font-family:Courier">x=numpy.fft.ifft(X)</span>, или наоборот, используя <span style="font-family:Courier">X=numpy.fft.fft(x)</span>.

### Решение задачи с помощью Python

Предположим теперь, что мы сохраняем звук поезда, когда он движется к наблюдателю, в векторе <span style="font-family:Courier">sample1</span>. Аналогично, мы сохраняем звук поезда, когда он удаляется от наблюдателя, в <span style="font-family:Courier">sample2</span>.

Мы вычисляем FFT этих сигналов и сохраняем их в векторах <span style="font-family:Courier">p1</span> и <span style="font-family:Courier">p2</span> соответственно.
    
    p1=fft(sample1)
    p2=fft(sample2)

Чтобы получить меру величины амплитуд, мы вычисляем их абсолютные значения в квадрате, элемент за элементом:

    P1=np.absolute(p1)**2
    P2=np.absolute(p2)**2

Наконец, мы вычисляем частоту, соответствующую каждому из элементов в <span style="font-family:Courier">P1</span> и <span style="font-family:Courier">P2.</span>

    f=linspace[0, N-1, N]/(N*dt)

Все детали, рассмотренные выше, реализованы в функции <span style="font-family:Courier">fftwrapper()</span>. Требуется еще несколько технических подробностей о том, как импортировать аудиофайл и т.д., Но это не будет обсуждаться здесь.  Тем не менее, вам рекомендуется ознакомиться с кодом и посмотреть, сможете ли вы разобраться в нем.

<!---The main thing you need to know for now is that <span style="font-family:Courier">fftwrapper</span> is called with an argument that can be true or false. This depends on whether you want to play the sound file and the samples <span style="font-family:Courier">sample</span>1 and <span style="font-family:Courier">sample2</span> or not. The return values are <span style="font-family:Courier">P1</span>, <span style="font-family:Courier">P2</span> and <span style="font-family:Courier">f</span>.--->

In [ ]:
%matplotlib inline
#from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, ifft
import scipy.io.wavfile as wav
from IPython.core.display import HTML, display

# Set common figure parameters:
newparams = {'axes.labelsize': 11, 'axes.linewidth': 1, 'savefig.dpi': 300, 
             'lines.linewidth': 1.0, 'figure.figsize': (8, 3),
             'ytick.labelsize': 10, 'xtick.labelsize': 10,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,
             'legend.fontsize': 10, 'legend.frameon': True, 
             'legend.handlelength': 1.5}
plt.rcParams.update(newparams)

In [ ]:
def fftwrapper():
    """
    Output
       P1      :   1D vector
       P2      :   1D vector
       f       :   1D vector
    
    Example usage
       (P1,P2,f) = fftwrapper()
    """
    
    N=80000        # Number of samplings in sample1 and sample2
    shift1=190000  # First index of sample1
    shift2=320000  # First index of sample2
    fcutoff=700    # Highest frequency in returned spectrum
    
    # Load sound file and convert from stereo to mono
    Fs, ystereo = wav.read('2wa_lwrhuntrd_ns197.wav', 'r')
    ymono = (ystereo[:,0] + ystereo[:,1])/2
    ymono = ymono/max(abs(ymono))
    deltat = 1/Fs
    
    sample1 = ymono[shift1:N+shift1-1]
    sample2 = ymono[shift2:N+shift2-1]
    
    # Do FFTs
    p1 = fft(sample1)
    p2 = fft(sample2)
    P1 = np.absolute(p1)**2
    P2 = np.absolute(p2)**2
    f = np.linspace(0,N-1,N)/(N*deltat)
    
    # Crop vectors to the sizes we are interested in
    ifcutoff= int(np.nonzero(abs(f-fcutoff)==min(abs(f-fcutoff)))[0]-1)
    f = f[0:ifcutoff]
    P1 = P1[0:ifcutoff]
    P2 = P2[0:ifcutoff]
    
    return (P1, P2, f)

Мы начинаем с вызова <span style="font-family:Courier">fftwrapper()</span>,

In [ ]:
(P1, P2, f) = fftwrapper()

Затем мы строим P1 и P2, нормализованные таким образом, что наибольшие элементы в построенных векторах равны 1.

In [ ]:
plt.plot(f, P1/max(P1), f, P2/max(P2))
plt.xlabel(r"$f$ (Hz)");
plt.ylabel(r"$P/P_{max}$")
plt.legend(['Sample 1','Sample 2','Location','NorthWest'], loc=2);

Эти два образца содержат пики частоты. Пики в <span style="font-family:Courier">sample2</span> смещены в сторону меньших частот по отношению к пикам в <span style="font-family:Courier">sample1</span>. Это согласуется с тем, что мы слышим, когда поезд проезжает мимо: звук, который он издает, удаляясь от нас, более низкий, чем звук, когда он движется к нам.

Мы выбираем частоту $f_1$, соответствующую самому высокому пику в <span style="font-family:Courier">sample1</span> на рисунке. Мы видим, что соответствующий пик в <span style="font-family:Courier">sample2</span> также является самым высоким, и мы обозначаем его частоту $f_2$. Мы находим $f_1$ и $f_2$ следующим образом.

In [ ]:
f1 = f[P1==max(P1)]
f2 = f[P2==max(P2)]

print("f1 = %f, f2 = %f" % (f1, f2))

Обратите внимание, что мы не могли заранее ожидать, что самый высокий пик в <span style="font-family:Courier">sample1</span> также был бы самым высоким в <span style="font-family:Courier">sample2</span>. В принципе, это не данность, и нам нужен был график, чтобы подтвердить это.

Найдя $f_1$ и $f_2$, мы можем вычислить $v$ с помощью (2). Мы определяем скорость звука $c$ равной 340.29 м/с и делаем следующее:

In [ ]:
c = 340.29              # Speed of sound [m/s]
v = (f1-f2)/(f1+f2)*c   # Speed of train [m/s]
v = 3.6*v               # Speed of train [km/h]

print("Скорость поезда %0.2f km/h." % v)

Чтобы запустить это на своем собственном компьютере, не забудьте загрузить звуковой файл [2wa_lwrhuntrd_ns197.wav](https://www.numfys.net/media/notebooks/files/2wa_lwrhuntrd_ns197.wav)$^1$ в тот же каталог, что и файл записной книжки IPython. Внимательно прослушайте звуковой файл. Звучит ли разумно, что поезд движется со скоростью 38.77 km/h?

___
$^1$ Любезно предоставлено Дэвидом Сафди и Грегом Лавуа из <span style="font-family:Courier">fwarailfan.net</span>